In [1]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os

username = "user"  # get this from the instructor
password = "grad5100user"  # get this from the instructor
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"  # get this from the instructor
port = 3306

In [2]:
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

In [3]:
def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result

In [4]:
chef_info = run_sql("describe chefdetails;", engine)
for x in chef_info[:10]:
    print(x)

('num', 'bigint', 'YES', 'MUL', None, '')
('name', 'text', 'YES', '', None, '')
('chef', 'text', 'YES', '', None, '')
('hometown', 'text', 'YES', '', None, '')
('city', 'text', 'YES', '', None, '')
('state', 'text', 'YES', '', None, '')
('age', 'double', 'YES', '', None, '')
('season', 'text', 'YES', '', None, '')
('seasonNumber', 'bigint', 'YES', '', None, '')
('series', 'text', 'YES', '', None, '')


In [5]:
judge_info = run_sql("describe judges;", engine)
for x in judge_info[:10]:
    print(x)

('num', 'bigint', 'YES', 'MUL', None, '')
('season', 'text', 'YES', '', None, '')
('seasonNumber', 'bigint', 'YES', '', None, '')
('series', 'text', 'YES', '', None, '')
('episode', 'bigint', 'YES', '', None, '')
('challengeType', 'text', 'YES', '', None, '')
('outcomeType', 'text', 'YES', '', None, '')
('guestJudge', 'text', 'YES', '', None, '')
('competedOnTC', 'text', 'YES', '', None, '')
('otherShows', 'text', 'YES', '', None, '')


### Q1

In [6]:
# removing seasons with 'Masters' and 'Canada 6' from the chefs dataset

chefdetails = run_sql("select * from chefdetails where substring(season,1,7) <> 'Masters' and substring(season,1,6) <> 'Canada';", engine)
#for x in chefdetails[:]:
#    print(x)

In [7]:
# removing seasons with 'Masters' and 'Canada 6' from the judges dataset

judges = run_sql("select * from judges where substring(season,1,7) <> 'Masters' and substring(season,1,6) <> 'Canada';", engine)
#for x in chefdetails[:]:
#    print(x)

In [8]:
chefdetails = pd.DataFrame(chefdetails)
judges = pd.DataFrame(judges)

In [9]:
chefdetails.sample()

,num,name,chef,hometown,city,state,age,season,seasonNumber,series,placement,personOfColor,occupation,gender
188,199,Carlos Fernandez,Carlos F.,None,Fort Lauderdale,Florida,36.0,Los Angeles,2,US,10,POC,None,Male


In [10]:
judges.sample()

,num,season,seasonNumber,series,episode,challengeType,outcomeType,guestJudge,competedOnTC,otherShows
15,15,All Stars: New York,8,US,8,Elimination,Individual,Anthony Bourdain,None,Parts Unknown; No Reservations


In [11]:
chefdetails['season'].unique()

array(['All Stars: New York', 'All-Stars L.A.', 'Boston', 'California',
       'Charleston', 'Chicago', 'D.C.', 'Colorado', 'Houston', 'Kentucky',
       'Las Vegas', 'Los Angeles', 'Miami', 'New Orleans', 'New York',
       'Portland', 'San Francisco', 'Seattle', 'Texas', 'World All Stars'],
      dtype=object)

In [12]:
judges['season'].unique()

array(['All Stars: New York', 'All-Stars L.A.', 'Boston', 'California',
       'Charleston', 'Charleston ', 'Chicago', 'D.C.', 'Colorado',
       'Houston', 'Kentucky', 'Las Vegas', 'Los Angeles', 'Miami',
       'New Orleans', 'New York', 'Portland', 'San Francisco', 'Seattle',
       'Texas', 'World All Stars'], dtype=object)

### Q2

In [13]:
chefdetails = chefdetails[['season', 'seasonNumber', 'name', 'placement', 'gender']]
chefdetails

,season,seasonNumber,name,placement,gender
0,All Stars: New York,8,Richard Blais,1,Male
1,All Stars: New York,8,Mike Isabella,2,Male
2,All Stars: New York,8,Antonia Lofaso,3,Female
3,All Stars: New York,8,Tiffany Derry,4,Female
4,All Stars: New York,8,Carla Hall,5,Female
...,...,...,...,...,...
317,World All Stars,20,May Phattanant Thongthong,14,Female
318,World All Stars,20,Begona Rodrigo,12,Female
319,World All Stars,20,Gabriel Rodriguez,2,Male
320,World All Stars,20,Amar Santana,6,Male


In [14]:
judges = judges[['season', 'seasonNumber', 'episode', 'challengeType', 'guestJudge', 'competedOnTC']]
judges

,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None
...,...,...,...,...,...,...
628,World All Stars,20,12,Quickfire,Sam Bompas,None
629,World All Stars,20,12,Elimination,Jeremy Chan,None
630,World All Stars,20,13,Quickfire,Greg Marchand,None
631,World All Stars,20,13,Elimination,Greg Marchand,None


### Q3

In [15]:
season_num = chefdetails[['season','seasonNumber']].copy().drop_duplicates().sort_values(by = 'seasonNumber').reset_index(drop = True)
season_num

,season,seasonNumber
0,San Francisco,1
1,Los Angeles,2
2,Miami,3
3,Chicago,4
4,New York,5
5,Las Vegas,6
6,D.C.,7
7,All Stars: New York,8
8,Texas,9
9,Seattle,10


a - Among all chef contestants, how many are male and how many are female?

In [16]:
chefdetails.groupby(['gender']).agg({'name': pd.Series.nunique})

,name
gender,
Female,128
Male,144


b - Among all winners (placement==1) how many are male and how many are female?

In [17]:
chefdetails[chefdetails['placement'] == 1].groupby(['gender']).agg({'name': pd.Series.nunique})

,name
gender,
Female,6
Male,13


c - Among all sets of top 3 finalists (placement=1,2,3) how many are male and how many
are female?

In [18]:
chefdetails.loc[(chefdetails['placement'] <= 3)].groupby(['gender']).agg({'name': pd.Series.nunique})

,name
gender,
Female,22
Male,32


d - Compare the number of times a female was placed in the top 3 with the number of times
a female won. Does this number seem unusual? That is, are women who place in the
top 3 less likely than expected to ultimately win the competition? Why or why not?

In [19]:
chefdetails[(chefdetails['gender'] == 'Female') & (chefdetails['placement'] <= 3)].shape[0]

25

In [20]:
chefdetails[(chefdetails['gender'] == 'Male') & (chefdetails['placement'] <= 3)].shape[0]

36

In [21]:
chefdetails[(chefdetails['gender'] == 'Female') & (chefdetails['placement'] == 1)].shape[0]

6

In [22]:
chefdetails[(chefdetails['gender'] == 'Male') & (chefdetails['placement'] == 1)].shape[0]

14

In [23]:
print('Ratio for males:', 14/36) #ratio for males
print('Ratio for females:', 6/25) #ratio for males

Ratio for males: 0.3888888888888889
Ratio for females: 0.24


When we compare the ratio of males that had placed in the top 3 and won to the ratio of females for the same condition, we see that ratio of women are 24% and the ratio for men are %39. This indicates that women are less likely to win the competition comparing to men.

4 - Make a table with one row for each first place winner containing the winner’s name,
the season they won, and a column containing ‘Yes’ or ‘No’ depending on whether they
served as a guest judge or not.

In [24]:
winners = chefdetails[chefdetails['placement'] == 1].merge(judges, left_on = 'name', right_on = 'guestJudge', how = 'left').drop_duplicates(subset = 'season_x').reset_index(drop = True)
winners = winners.rename(columns = {'season_x':'season'})
winners = winners[['name','season','guestJudge']]
winners['guestJudge'] = winners['guestJudge'].apply(lambda x: 'No' if pd.isna(x) else 'Yes')
winners

,name,season,guestJudge
0,Richard Blais,All Stars: New York,Yes
1,Melissa King,All-Stars L.A.,Yes
2,Mei Lin,Boston,No
3,Jeremy Ford,California,No
4,Brooke Williamson,Charleston,Yes
5,Stephanie Izard,Chicago,Yes
6,Kevin Sbraga,D.C.,No
7,Joseph Flamm,Colorado,No
8,Kah-wai Lo,Houston,No
9,Kelsey Barnard Clark,Kentucky,Yes


5 - Find all contestants who were not first place winners but did serve as a guest judge at
some point. Make sure to only list each name once.

In [25]:
notwinners = chefdetails[chefdetails['placement'] != 1].merge(judges, left_on = 'name', right_on = 'guestJudge', how = 'inner').drop_duplicates(subset = 'name').reset_index()
notwinners = notwinners[['name']]
notwinners

,name
0,Mike Isabella
1,Antonia Lofaso
2,Tiffany Derry
3,Dale Talde
4,Spike Mendelsohn
5,Stephen Asprinio
6,Bryan Voltaggio
7,Gregory Gourdet
8,Karen Akunowicz
9,Lee Anne Wong


In [26]:
intersection = notwinners[notwinners['name'].isin(winners['name'])]
intersection

,name
11,Melissa King
16,Richard Blais
25,Brooke Williamson


In [27]:
notwinners_filtered = notwinners[~notwinners['name'].isin(intersection['name'])].reset_index(drop = True)
notwinners_filtered

,name
0,Mike Isabella
1,Antonia Lofaso
2,Tiffany Derry
3,Dale Talde
4,Spike Mendelsohn
5,Stephen Asprinio
6,Bryan Voltaggio
7,Gregory Gourdet
8,Karen Akunowicz
9,Lee Anne Wong
